# Marine Regions LD Feed

Exploring working with the data from these commands

## Current thoughts

* Getting the data graph is a simple call and then we can frame it and use similar methods to get our URL set.
* At that point it's the classic Gleaner workflow, no changes seen.  
* As we get new feed updatges we can leverage the prov graph generated by Gleaner combined with the feed time stamps (nothing hard there) 
* This is a nice way to maintain updates in the OIH graph vs the prov pattern (complements likely, not replaces)|
* put the feed URL in the robots.txt?

## Data graphs via curl

```bash
curl -L --url http://marineregions.org/mrgid/2795.jsonld
curl -L --url http://marineregions.org/mrgid/2795 -H Accept:application/ld+json
curl -L --url http://marineregions.org/feed -HAccept:application/ld+json
```

## refs:

- [LDF](linkdatafragments.org)
- [LDES specification](https://w3id.org/ldes/specification)
- [TREE](https://treecg.github.io/specification/) and the [TREEcg repo](https://github.com/TREEcg/specification)
- https://www.marineregions.org/gazetteer.php?p=webservices


## my installs and imports..   

In [1]:
!pip install -q 'PyLD>=2.0.3'

In [2]:
from pyld import jsonld
import json

## Problems

In the following, if I frame with "framenc", I get results like:

```json
{
            "@id": "mrgid/103?t=1626519699",
            "dcterms:isVersionOf": {
                "@id": "mrgid/103"
            },
            "dcterms:modified": {
                "@type": "http://www.w3.org/2001/XMLSchema#dateTime",
                "@value": "2021-07-17T11:01:39Z"
            }
        },
```

which lack the fulled express domain name based @id values.  I was hoping to get results with a full qualified domain name from the frame and then I would simply place that into a native Go struct that I could pass along to my regular indexing code without changes.  

If I use "frame" which should be the full context.  I get an error "list index out of range"

Also, why does the context contain both

*       "dcterms": "http://purl.org/dc/terms/",
*       "dct": "http://purl.org/dc/terms//dup/",

Both are used in the data graph..   when I tried to align them though, I got some other errors..  likely I had an error in the context.

In [3]:
docurl = "http://marineregions.org/feed.jsonld"

# with open('./data/mregFeedv2.json', 'rb') as f:
#     doc = f.read()

# this is not used..   had it for when playing with compact and expand
context =  {
      "tree": "https://w3id.org/tree#",
      "ldes": "https://w3id.org/ldes#",
      "ldes:versionKey": {
        "@type": "@id",
        "@container": "@list"
      },
      "dcterms": "http://purl.org/dc/terms/",
      "dcterms:isVersionOf": {
        "@type": "@id"
      },
      "dcterms:modified": {
        "@type": "xsd:dateTime"
      },
      "xsd": "http://www.w3.org/2001/XMLSchema#",
      "sh": "http://www.w3.org/ns/shacl#",
      "schema": "https://schema.org/",
      "skos": "http://www.w3.org/2004/02/skos/core#",
      "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
      "mr": "http://marineregions.org/ns/ontology#",
      "dct": "http://purl.org/dc/terms//dup/",
      "gsp": "http://www.opengis.net/ont/geosparql#",
      "dcat": "http://www.w3.org/ns/dcat#"
    }

frame = {
  "@context": {
      "tree": "https://w3id.org/tree#",
      "ldes": "https://w3id.org/ldes#",
      "ldes:versionKey": {
        "@type": "@id",
        "@container": "@list"
      },
      "dcterms": "http://purl.org/dc/terms/",
      "dcterms:isVersionOf": {
        "@type": "@id"
      },
      "dcterms:modified": {
        "@type": "xsd:dateTime"
      },
      "xsd": "http://www.w3.org/2001/XMLSchema#",
      "sh": "http://www.w3.org/ns/shacl#",
      "schema": "https://schema.org/",
      "skos": "http://www.w3.org/2004/02/skos/core#",
      "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
      "mr": "http://marineregions.org/ns/ontology#",
      "dct": "http://purl.org/dc/terms//dup/",
      "gsp": "http://www.opengis.net/ont/geosparql#",
      "dcat": "http://www.w3.org/ns/dcat#"
    },
  "@explicit": "true",
  "dcterms:isVersionOf": "",
  "dcterms:modified": ""
}

framenc = {
  "@context": {"schema": "https://schema.org/",
               "dcterms": "http://purl.org/dc/terms/"},
  "@explicit": "true",
  "dcterms:isVersionOf": "",
  "dcterms:modified": ""
}

# rg = jsonld.expand(doc, context)  # would ned to define context for this code
rg = jsonld.frame(docurl, framenc)  ##  framenc: works..   but no FQDN.    frame:  gives errors

jd = json.dumps(rg, indent=4)
print(jd)


{
    "@context": {
        "schema": "https://schema.org/",
        "dcterms": "http://purl.org/dc/terms/"
    },
    "@graph": [
        {
            "@id": "mrgid/1053?t=1629807612",
            "dcterms:isVersionOf": {
                "@id": "mrgid/1053"
            },
            "dcterms:modified": {
                "@type": "http://www.w3.org/2001/XMLSchema#dateTime",
                "@value": "2021-08-24T12:20:12Z"
            }
        },
        {
            "@id": "mrgid/63408?t=1629807612",
            "dcterms:isVersionOf": {
                "@id": "mrgid/63408"
            },
            "dcterms:modified": {
                "@type": "http://www.w3.org/2001/XMLSchema#dateTime",
                "@value": "2021-08-24T12:20:12Z"
            }
        }
    ]
}
